In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import json
import pandas as pd
from pathlib import Path
from copy import deepcopy
from functools import partial

from bellek.qa.ablation import answer_question_standard, answer_question_cte
from bellek.utils import set_seed, jprint
from bellek.musique.singlehop import benchmark as benchmark_single
from bellek.musique.multihop import benchmark as benchmark_multi

set_seed(89)

/home/pc/anaconda3/envs/baris/lib/python3.10/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [3]:
pd.options.display.float_format = '{:,.3f}'.format

In [4]:
from tqdm.auto import tqdm
tqdm.pandas()

In [5]:
report_df = pd.read_json("./our-method-report-20240726-001458.jsonl", orient='records', lines=True)
report_df.head()

,qdecomp,context,retrieval,top_k,qa,run,exact_match,f1
0,False,paragraphs,bm25,3,standard,1,0.190,0.287
1,False,paragraphs,semantic,3,standard,1,0.280,0.372
2,False,paragraphs,bm25,5,standard,1,0.250,0.358
3,False,paragraphs,semantic,5,standard,1,0.270,0.393
4,False,paragraphs,bm25,10,standard,1,0.260,0.386


In [6]:
manual_df = pd.DataFrame.from_records([
    {"qdecomp": True, "context": "triplets", "retrieval": "graph-search", "top_k": 10, "qa": "standard", "exact_match": 0.440, "f1": 0.519, "run": 1},
{"qdecomp": True, "context": "paragraphs+triplets", "retrieval": "graph-search", "top_k": 10, "qa": "standard", "exact_match": 0.510, "f1": 0.604, "run": 1}
])
manual_df.head()

,qdecomp,context,retrieval,top_k,qa,exact_match,f1,run
0,True,triplets,graph-search,10,standard,0.440,0.519,1
1,True,paragraphs+triplets,graph-search,10,standard,0.510,0.604,1


In [7]:
all_report_df = pd.concat([report_df, manual_df], axis=0).reset_index(drop=True)
all_report_df

,qdecomp,context,retrieval,top_k,qa,run,exact_match,f1
0,False,paragraphs,bm25,3,standard,1,0.190,0.287
1,False,paragraphs,semantic,3,standard,1,0.280,0.372
2,False,paragraphs,bm25,5,standard,1,0.250,0.358
3,False,paragraphs,semantic,5,standard,1,0.270,0.393
4,False,paragraphs,bm25,10,standard,1,0.260,0.386
...,...,...,...,...,...,...,...,...
85,True,triplets,semantic,9,standard,2,0.400,0.495
86,True,triplets,bm25,15,standard,2,0.430,0.526
87,True,triplets,semantic,15,standard,2,0.450,0.531
88,True,triplets,graph-search,10,standard,1,0.440,0.519


In [8]:
all_report_df.rename(
    columns={
        'qdecomp': 'Q-decomp',
        'context': 'Context',
        'retrieval': 'Retrieval',
        'top_k': 'top-k',
        'qa': 'QA Prompting',
        'run': 'Run',
        'exact_match': "EM",
        'f1': "F1",
    }, 
    inplace=True,
)

In [9]:
avg_report_df = all_report_df.drop(columns=['Run']).groupby(['Q-decomp', 'Context', 'QA Prompting', 'Retrieval', 'top-k']).mean()
avg_report_df

EM    F1
Q-decomp Context             QA Prompting Retrieval    top-k            
False    paragraphs          cte          bm25         3     0.170 0.269
                                                       5     0.200 0.308
                                                       10    0.280 0.388
                                          semantic     3     0.270 0.378
                                                       5     0.275 0.373
                                                       10    0.310 0.403
                             standard     bm25         3     0.190 0.293
                                                       5     0.250 0.355
                                                       10    0.265 0.391
                                          semantic     3     0.290 0.380
                                                       5     0.275 0.392
                                                       10    0.285 0.419
         paragraphs+triplets standard     bm25         3     0.220 0.368
                                                       5     0.220 0.371
                                                       10    0.220 0.366
                                          semantic     3     0.190 0.326
                                                       5     0.190 0.326
                                                       10    0.195 0.326
         triplets            standard     bm25         15    0.225 0.351
                                                       25    0.230 0.345
                                          semantic     15    0.205 0.321
                                                       25    0.240 0.359
True     paragraphs          cte          bm25         3     0.510 0.625
                                                       5     0.490 0.591
                                                       10    0.560 0.637
                                          semantic     3     0.540 0.624
                                                       5     0.555 0.661
                                                       10    0.520 0.623
                             standard     bm25         3     0.535 0.619
                                                       5     0.545 0.628
                                                       10    0.535 0.623
                                          semantic     3     0.545 0.635
                                                       5     0.525 0.609
                                                       10    0.540 0.626
         paragraphs+triplets standard     bm25         3     0.500 0.573
                                                       5     0.510 0.588
                                                       10    0.510 0.594
                                          graph-search 10    0.510 0.604
                                          semantic     3     0.510 0.609
                                                       5     0.525 0.622
                                                       10    0.520 0.617
         triplets            standard     bm25         9     0.360 0.456
                                                       15    0.425 0.518
                                          graph-search 10    0.440 0.519
                                          semantic     9     0.405 0.499
                                                       15    0.450 0.529

In [12]:
print(avg_report_df.to_latex())

\begin{tabular}{lllllrr}
\toprule
 &  &  &  &  & EM & F1 \\
Q-decomp & Context & QA Prompting & Retrieval & top-k &  &  \\
\midrule
\multirow[t]{22}{*}{False} & \multirow[t]{12}{*}{paragraphs} & \multirow[t]{6}{*}{cte} & \multirow[t]{3}{*}{bm25} & 3 & 0.170000 & 0.268718 \\
 &  &  &  & 5 & 0.200000 & 0.308480 \\
 &  &  &  & 10 & 0.280000 & 0.388278 \\
\cline{4-7}
 &  &  & \multirow[t]{3}{*}{semantic} & 3 & 0.270000 & 0.378173 \\
 &  &  &  & 5 & 0.275000 & 0.373259 \\
 &  &  &  & 10 & 0.310000 & 0.402842 \\
\cline{3-7} \cline{4-7}
 &  & \multirow[t]{6}{*}{standard} & \multirow[t]{3}{*}{bm25} & 3 & 0.190000 & 0.292683 \\
 &  &  &  & 5 & 0.250000 & 0.355292 \\
 &  &  &  & 10 & 0.265000 & 0.391366 \\
\cline{4-7}
 &  &  & \multirow[t]{3}{*}{semantic} & 3 & 0.290000 & 0.379906 \\
 &  &  &  & 5 & 0.275000 & 0.391969 \\
 &  &  &  & 10 & 0.285000 & 0.419140 \\
\cline{2-7} \cline{3-7} \cline{4-7}
 & \multirow[t]{6}{*}{paragraphs+triplets} & \multirow[t]{6}{*}{standard} & \multirow[t]{3}{*}{bm25}

In [13]:
avg_report_df.sort_values('EM')

EM    F1
Q-decomp Context             QA Prompting Retrieval    top-k            
False    paragraphs          cte          bm25         3     0.170 0.269
         paragraphs+triplets standard     semantic     5     0.190 0.326
                                                       3     0.190 0.326
         paragraphs          standard     bm25         3     0.190 0.293
         paragraphs+triplets standard     semantic     10    0.195 0.326
         paragraphs          cte          bm25         5     0.200 0.308
         triplets            standard     semantic     15    0.205 0.321
         paragraphs+triplets standard     bm25         10    0.220 0.366
                                                       5     0.220 0.371
                                                       3     0.220 0.368
         triplets            standard     bm25         15    0.225 0.351
                                                       25    0.230 0.345
                                          semantic     25    0.240 0.359
         paragraphs          standard     bm25         5     0.250 0.355
                                                       10    0.265 0.391
                             cte          semantic     3     0.270 0.378
                             standard     semantic     5     0.275 0.392
                             cte          semantic     5     0.275 0.373
                                          bm25         10    0.280 0.388
                             standard     semantic     10    0.285 0.419
                                                       3     0.290 0.380
                             cte          semantic     10    0.310 0.403
True     triplets            standard     bm25         9     0.360 0.456
                                          semantic     9     0.405 0.499
                                          bm25         15    0.425 0.518
                                          graph-search 10    0.440 0.519
                                          semantic     15    0.450 0.529
         paragraphs          cte          bm25         5     0.490 0.591
         paragraphs+triplets standard     bm25         3     0.500 0.573
                                                       10    0.510 0.594
                                          semantic     3     0.510 0.609
                                          graph-search 10    0.510 0.604
                                          bm25         5     0.510 0.588
         paragraphs          cte          bm25         3     0.510 0.625
         paragraphs+triplets standard     semantic     10    0.520 0.617
         paragraphs          cte          semantic     10    0.520 0.623
                             standard     semantic     5     0.525 0.609
         paragraphs+triplets standard     semantic     5     0.525 0.622
         paragraphs          standard     bm25         10    0.535 0.623
                                                       3     0.535 0.619
                             cte          semantic     3     0.540 0.624
                             standard     semantic     10    0.540 0.626
                                                       3     0.545 0.635
                                          bm25         5     0.545 0.628
                             cte          semantic     5     0.555 0.661
                                          bm25         10    0.560 0.637

In [14]:
avg_report_df.sort_values('F1')

EM    F1
Q-decomp Context             QA Prompting Retrieval    top-k            
False    paragraphs          cte          bm25         3     0.170 0.269
                             standard     bm25         3     0.190 0.293
                             cte          bm25         5     0.200 0.308
         triplets            standard     semantic     15    0.205 0.321
         paragraphs+triplets standard     semantic     10    0.195 0.326
                                                       5     0.190 0.326
                                                       3     0.190 0.326
         triplets            standard     bm25         25    0.230 0.345
                                                       15    0.225 0.351
         paragraphs          standard     bm25         5     0.250 0.355
         triplets            standard     semantic     25    0.240 0.359
         paragraphs+triplets standard     bm25         10    0.220 0.366
                                                       3     0.220 0.368
                                                       5     0.220 0.371
         paragraphs          cte          semantic     5     0.275 0.373
                                                       3     0.270 0.378
                             standard     semantic     3     0.290 0.380
                             cte          bm25         10    0.280 0.388
                             standard     bm25         10    0.265 0.391
                                          semantic     5     0.275 0.392
                             cte          semantic     10    0.310 0.403
                             standard     semantic     10    0.285 0.419
True     triplets            standard     bm25         9     0.360 0.456
                                          semantic     9     0.405 0.499
                                          bm25         15    0.425 0.518
                                          graph-search 10    0.440 0.519
                                          semantic     15    0.450 0.529
         paragraphs+triplets standard     bm25         3     0.500 0.573
                                                       5     0.510 0.588
         paragraphs          cte          bm25         5     0.490 0.591
         paragraphs+triplets standard     bm25         10    0.510 0.594
                                          graph-search 10    0.510 0.604
                                          semantic     3     0.510 0.609
         paragraphs          standard     semantic     5     0.525 0.609
         paragraphs+triplets standard     semantic     10    0.520 0.617
         paragraphs          standard     bm25         3     0.535 0.619
         paragraphs+triplets standard     semantic     5     0.525 0.622
         paragraphs          cte          semantic     10    0.520 0.623
                             standard     bm25         10    0.535 0.623
                             cte          semantic     3     0.540 0.624
                                          bm25         3     0.510 0.625
                             standard     semantic     10    0.540 0.626
                                          bm25         5     0.545 0.628
                                          semantic     3     0.545 0.635
                             cte          bm25         10    0.560 0.637
                                          semantic     5     0.555 0.661

In [18]:
with open("experiment-results-agg.tex", 'w') as f:
    f.write(avg_report_df.to_latex())

In [22]:
with open("experiment-results-detailed.tex", 'w') as f:
    f.write(all_report_df.sort_values(['Q-decomp', 'Context', 'QA Prompting', 'Retrieval', 'top-k', 'Run']).to_latex(index=False))